## Keras Layers Practice

https://keras.io/api/layers/core_layers/embedding/

https://keras.io/api/layers/recurrent_layers/lstm/

In [2]:
import tensorflow as tf
import numpy as np

np.random.seed(42)

### Embedding Layer

In [38]:
model = tf.keras.Sequential()
embedding_layer = tf.keras.layers.Embedding(input_dim=1000,
                                            output_dim=64,
                                            input_length=256)
model.add(embedding_layer)

In [39]:
input_array = np.random.randint(1000, size=(1000, 256))
input_array.shape
#64, 14, 256

(1000, 256)

In [40]:
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
output_array.shape

32/32 [==============================] - 0s 3ms/step


(1000, 256, 64)

### LSTM Layer

In [41]:
inputs = np.random.random((32, 10, 256))

In [42]:
lstm = tf.keras.layers.LSTM(4)
output = lstm(inputs)
output.shape

TensorShape([32, 4])

In [43]:
lstm = tf.keras.layers.LSTM(4,
                         return_sequences=True,
                         return_state=True)
whole_seq_output, final_memory_state, final_carry_state = lstm(inputs)
whole_seq_output.shape

TensorShape([32, 10, 4])

In [44]:
final_memory_state.shape

TensorShape([32, 4])

In [45]:
final_carry_state.shape

TensorShape([32, 4])

### Embedding to LSTM Layer

In [84]:
max_sequence_len = 45
vocab_size = 1000
lstm_units = 12

In [85]:
input = np.random.randint(1000, size=(64, max_sequence_len))
input.shape

(64, 45)

In [86]:
embedding = tf.keras.layers.Embedding(name="embedding",
                                      input_dim=vocab_size,
                                      output_dim=lstm_units,
                                      input_length=max_sequence_len)
lstm = tf.keras.layers.LSTM(name="LSTM",
                            units=lstm_units)
model2 = tf.keras.Sequential()
model2.add(embedding)
model2.add(lstm)
model2.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 12)            12000     
                                                                 
 LSTM (LSTM)                 (None, 12)                1200      
                                                                 
Total params: 13200 (51.56 KB)
Trainable params: 13200 (51.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [87]:
model2.compile('rmsprop', 'mse')
output_array = model2.predict(input)
output_array.shape

2/2 [==============================] - 0s 11ms/step


(64, 12)

In [88]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, units):
        """_summary_

        Args:
            vocab_size (_type_): _description_
            units (_type_): _description_
        """
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(name="embedding",
                                      input_dim=vocab_size,
                                      output_dim=units)
        self.lstm = tf.keras.layers.LSTM(name="LSTM",
                                    units=units)

    def call(self, context):
        """_summary_

        Args:
            context (_type_): _description_
            target (_type_): _description_
        """
        x = self.embedding(context)
        x = self.lstm(x)
        return x

In [89]:
decoder = Decoder(vocab_size, lstm_units)

# Notice that you don't need the embedded version of sr_translation since this is done inside the class
logits = decoder(input)